<a href="https://colab.research.google.com/github/bonillahermes/Data_Science_Projects/blob/main/EDA6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hermes Yate Bonilla
**Data Scientist**
---

**Contact:**
- **Email:** [bonillahermes@gmail.com](mailto:bonillahermes@gmail.com)
- **LinkedIn:** [linkedin.com/in/bonillahermes](https://www.linkedin.com/in/bonillahermes/)
- **GitHub:** [github.com/bonillahermes](https://github.com/bonillahermes)
- **Webpage:** [bonillahermes.com](https://bonillahermes.com/)
---

# Análisis Exploratorio

In [ ]:
# Instalación de librerías
!pip install keras_tuner

In [ ]:
# Importación de librerías generales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import skew, kurtosis
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.stats import f_oneway
from scipy.spatial.distance import mahalanobis
from scipy.stats import kruskal

# Importación de bibliotecas de machine learning de scikit-learn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, cross_val_score
from sklearn.metrics import (
    accuracy_score, confusion_matrix, precision_score, recall_score,
    roc_curve, roc_auc_score, classification_report, precision_recall_curve, average_precision_score,
    mean_squared_error, r2_score, silhouette_score
)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

# Importación de XGBoost
import xgboost as xgb

# Importación de bibliotecas para modelos y redes neuronales
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Importación de Keras Tuner
import keras_tuner as kt

# Importación de bibliotecas para visualización interactiva
import plotly.graph_objs as go
import plotly.express as px

# Análisis Descriptivo

In [ ]:
# Cargar las hojas del archivo Excel
file_path = 'Base_Final.xlsx'  # Asegúrate de subir el archivo a Colab o especificar la ruta correcta
df = pd.read_excel(file_path)

In [ ]:
# Recodificar la variable I2R5C1: 1 se mantiene igual, 2 y 3 se recodifican a 0
df['I2R5C1'] = df['I2R5C1'].replace({2: 0, 3: 0})

In [ ]:
# Redondear los valores de la variable I2R5C1 a enteros
df['I2R5C1'] = df['I2R5C1'].round().astype(int)

In [ ]:
# Redondear los valores de la variable I10R7C1 a enteros
df['I10R7C1'] = df['I10R7C1'].round().astype(int)

In [ ]:
# Mostrar la dimensión del Dataframe
df.shape

In [ ]:
# Mostrar el número de valores faltantes por columna
df.isnull().sum()

In [ ]:
# Asegúrate de que 'df' contiene las variables necesarias
variables_descriptivas = ['II1R3C2', 'I3R2C1']

# Filtrar el DataFrame para dejar solo las variables de interés
df_descriptivo = df[variables_descriptivas]

In [ ]:
# Obtener estadísticos descriptivos básicos
estadisticos_descriptivos = df_descriptivo.describe()

# Calcular coeficiente de variación, asimetría, curtosis y rango intercuartílico
coef_var = df_descriptivo.std() / df_descriptivo.mean()
asimetria = df_descriptivo.apply(skew)
curtosis = df_descriptivo.apply(kurtosis)
iqr = df_descriptivo.quantile(0.75) - df_descriptivo.quantile(0.25)

# Crear un DataFrame con los estadísticos adicionales
estadisticos_adicionales = pd.DataFrame({
    'Coeficiente de Variación': coef_var,
    'Asimetría': asimetria,
    'Curtosis': curtosis,
    'Rango Intercuartílico': iqr
})

# Combinar los estadísticos básicos y adicionales
estadisticos_completos = pd.concat([estadisticos_descriptivos, estadisticos_adicionales.T])

# Mostrar los estadísticos descriptivos completos
print("Estadísticos Descriptivos Completos:")
print(estadisticos_completos)

In [ ]:
# Función para calcular estadísticos descriptivos para cada grupo de cuartiles
def calcular_estadisticos_por_cuartiles(df, columna):
    cuartiles = df[columna].quantile([0.25, 0.5, 0.75])
    grupos = {
        'min-Q1': df[df[columna] <= cuartiles[0.25]],
        'Q1-Q2': df[(df[columna] > cuartiles[0.25]) & (df[columna] <= cuartiles[0.5])],
        'Q2-Q3': df[(df[columna] > cuartiles[0.5]) & (df[columna] <= cuartiles[0.75])],
        'Q3-max': df[df[columna] > cuartiles[0.75]]
    }
    estadisticos = {}
    for nombre, grupo in grupos.items():
        descrip = grupo[columna].describe()
        descrip['Coeficiente de Variación'] = grupo[columna].std() / grupo[columna].mean()
        descrip['Asimetría'] = skew(grupo[columna])
        descrip['Curtosis'] = kurtosis(grupo[columna])
        estadisticos[nombre] = descrip
    return pd.DataFrame(estadisticos)

# Calcular los estadísticos descriptivos por cuartiles para cada variable
estadisticos_cuartiles_II1R3C2 = calcular_estadisticos_por_cuartiles(df, 'II1R3C2')
estadisticos_cuartiles_I3R2C1 = calcular_estadisticos_por_cuartiles(df, 'I3R2C1')

# Mostrar los estadísticos descriptivos por cuartiles
print("Estadísticos Descriptivos por Cuartiles para II1R3C2:")
print(estadisticos_cuartiles_II1R3C2)

print("\nEstadísticos Descriptivos por Cuartiles para I3R2C1:")
print(estadisticos_cuartiles_I3R2C1)

In [ ]:
# Crear boxplot para las variables II1R3C2 e I3R2C1
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.boxplot(df['II1R3C2'])
plt.title('Boxplot de II1R3C2')
plt.ylabel('Valores')

plt.subplot(1, 2, 2)
plt.boxplot(df['I3R2C1'])
plt.title('Boxplot de I3R2C1')
plt.ylabel('Valores')

plt.tight_layout()
plt.show()


In [ ]:
# Crear histograma para las variables II1R3C2 e I3R2C1
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(df['II1R3C2'], bins=30, edgecolor='black')
plt.title('Histograma de II1R3C2')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')

plt.subplot(1, 2, 2)
plt.hist(df['I3R2C1'], bins=30, edgecolor='black')
plt.title('Histograma de I3R2C1')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')

plt.tight_layout()
plt.show()


In [ ]:
# Crear boxplot de II1R3C2 basado en las categorías de I2R5C1
plt.figure(figsize=(12, 6))

df.boxplot(column='II1R3C2', by='I2R5C1')
plt.title('Boxplot de II1R3C2 por I2R5C1')
plt.suptitle('')
plt.xlabel('I2R5C1')
plt.ylabel('II1R3C2')



In [ ]:
# Crear boxplot de I3R2C1 basado en las categorías de I2R5C1
df.boxplot(column='I3R2C1', by='I2R5C1')
plt.title('Boxplot de I3R2C1 por I2R5C1')
plt.suptitle('')
plt.xlabel('I2R5C1')
plt.ylabel('I3R2C1')

plt.tight_layout()
plt.show()

In [ ]:
# Crear boxplot de II1R3C2 basado en las categorías de I10R7C1
plt.figure(figsize=(12, 6))

df.boxplot(column='II1R3C2', by='I10R7C1')
plt.title('Boxplot de II1R3C2 por I10R7C1')
plt.suptitle('')
plt.xlabel('I10R7C1')
plt.ylabel('II1R3C2')

In [ ]:
# Crear boxplot de I3R2C1 basado en las categorías de I10R7C1
df.boxplot(column='I3R2C1', by='I10R7C1')
plt.title('Boxplot de I3R2C1 por I10R7C1')
plt.suptitle('')
plt.xlabel('I10R7C1')
plt.ylabel('I3R2C1')

plt.tight_layout()
plt.show()

## Análisis Inferencial

In [ ]:
# Prueba de Kruskal-Wallis para II1R3C2 por I2R5C1
groups_II1R3C2_I2R5C1 = [df[df['I2R5C1'] == category]['II1R3C2'] for category in df['I2R5C1'].unique()]
kruskal_result_II1R3C2_I2R5C1 = kruskal(*groups_II1R3C2_I2R5C1)

print(f'Prueba de Kruskal-Wallis para II1R3C2 por I2R5C1: H={kruskal_result_II1R3C2_I2R5C1.statistic}, p-value={kruskal_result_II1R3C2_I2R5C1.pvalue}')

In [ ]:
# Prueba de Kruskal-Wallis para I3R2C1 por I2R5C1
groups_I3R2C1_I2R5C1 = [df[df['I2R5C1'] == category]['I3R2C1'] for category in df['I2R5C1'].unique()]
kruskal_result_I3R2C1_I2R5C1 = kruskal(*groups_I3R2C1_I2R5C1)

print(f'Prueba de Kruskal-Wallis para I3R2C1 por I2R5C1: H={kruskal_result_I3R2C1_I2R5C1.statistic}, p-value={kruskal_result_I3R2C1_I2R5C1.pvalue}')

In [ ]:
# Prueba de Kruskal-Wallis para II1R3C2 por I10R7C1
groups_II1R3C2_I10R7C1 = [df[df['I10R7C1'] == category]['II1R3C2'] for category in df['I10R7C1'].unique()]
kruskal_result_II1R3C2_I10R7C1 = kruskal(*groups_II1R3C2_I10R7C1)

print(f'Prueba de Kruskal-Wallis para II1R3C2 por I10R7C1: H={kruskal_result_II1R3C2_I10R7C1.statistic}, p-value={kruskal_result_II1R3C2_I10R7C1.pvalue}')

In [ ]:
# Prueba de Kruskal-Wallis para I3R2C1 por I10R7C1
groups_I3R2C1_I10R7C1 = [df[df['I10R7C1'] == category]['I3R2C1'] for category in df['I10R7C1'].unique()]
kruskal_result_I3R2C1_I10R7C1 = kruskal(*groups_I3R2C1_I10R7C1)

print(f'Prueba de Kruskal-Wallis para I3R2C1 por I10R7C1: H={kruskal_result_I3R2C1_I10R7C1.statistic}, p-value={kruskal_result_I3R2C1_I10R7C1.pvalue}')